In [2]:
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [5]:
# 데이터 로드
article_df = pd.read_csv("article.csv")
article_df.tail(2)

,category,content,link,title
209,105,갤럭시폴드를 앞세운 삼성전자가 오는 22일 화웨이와 폴더블폰 정면 대결을 벌인다. ...,https://news.naver.com/main/read.nhn?mode=LSD&...,[TF초점] '갤럭시폴드' 中 6차 판매 관심 쏠리는 이유
210,105,20일 열린 기자간담회에서 김훈 개발실장이 퍼플에 대해서 설명하고 있다. /최승진 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,리니지2M이 퍼플과 만나니…'PC게임 부럽지 않네'


In [7]:
# 데이터 전처리 : 데이터 셋 분리
X_train, X_test, y_train, y_test = train_test_split(
article_df.content, article_df.category, test_size=0.1, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(189, 22, 189, 22)

In [10]:
#모델 만들기
clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB(alpha=0.1)),
])

In [11]:
#학습
model = clf.fit(X_train.values.astype("str"), y_train)

In [12]:
#모델 성능 확인
y_pred = model.predict(X_test)

In [15]:
list(y_test[:5]), list(y_pred[:5])

([100, 104, 100, 105, 100], [100, 104, 100, 105, 100])

In [16]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         100       0.93      1.00      0.96        13
         101       1.00      1.00      1.00         2
         102       1.00      0.50      0.67         2
         103       1.00      1.00      1.00         1
         104       1.00      1.00      1.00         1
         105       1.00      1.00      1.00         3

    accuracy                           0.95        22
   macro avg       0.99      0.92      0.94        22
weighted avg       0.96      0.95      0.95        22



### 모델 사용하기

In [17]:
categories = {
    100: "정치",
    101 : "경제",
    102 : "사회",
    103 : "생활/문화",
    102 : "세계",
    105 :  "IT/과학",
}

In [20]:
contents = [
    "네이버와 카카오는드론 기술 발전에 주력",
    "요즘 환율 주가 예측이 불가",
    "광화문 대한 애국당 천막, 격렬 저항 속 철거",
]

datas = {
    "content" : contents, 
    "category_code" : model.predict(contents),
}
df = pd.DataFrame(datas)
df

,content,category_code
0,네이버와 카카오는드론 기술 발전에 주력,101
1,요즘 환율 주가 예측이 불가,100
2,"광화문 대한 애국당 천막, 격렬 저항 속 철거",100


In [29]:
df['category'] = df["category_code"].apply(lambda data: categories[data])
df['proba'] = df["content"].apply(lambda data: round(max(model.predict_proba([data])[0]),2))

In [30]:
df

,content,category_code,category,proba
0,네이버와 카카오는드론 기술 발전에 주력,101,경제,0.41
1,요즘 환율 주가 예측이 불가,100,정치,0.33
2,"광화문 대한 애국당 천막, 격렬 저항 속 철거",100,정치,0.51


In [31]:
#모델저장

In [33]:
pickle.dump(model, open("clf.pkl", "wb"))